In [ ]:
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from scipy.fft import fft
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/project

/content/drive/MyDrive/project


In [ ]:
L=8064
Fs=128
Features=np.zeros((25600,32,3*128))
Labels=list()
for subject_id in range(0,32):
    if subject_id % 10 ==0:
      print(subject_id)
    loaded_data=None
    file_name=None
    if subject_id<9:
        file_name= 'data_preprocessed_python/s0'+str(subject_id+1)+'.dat'
    else:
        file_name= 'data_preprocessed_python/s'+str(subject_id+1)+'.dat'
    with open(file_name, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        loaded_data = u.load()
    data=loaded_data['data']
    Labels.append(loaded_data['labels'])
    
    for i in range(len(data)):
        for k in range(3,61,3):
          for j in range(32):   
            x=np.array(data[i][j][k*128:(k+3)*128]) -np.array((data[i][j][:3*128]))
            x=zscore(x)
            Features[subject_id*800+i*20+int(k/3-1)][j]=x

0
10
20
30


In [ ]:
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   

def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5):
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(40, (1, 13), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                          kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 35), strides=(1, 7))(block1)
    block1       = Activation(log)(block1)
    block1       = Dropout(dropoutRate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)

In [ ]:
y=np.zeros(800*32)
for j in range(32):
  for i in range(40):
    if Labels[j][i][0]>5:
      y[j*800+i*20:j*800+(i+1)*20]=1

In [ ]:
def run(train_index,test_index,
        dropout_rate=0.5):



  Y=np_utils.to_categorical(y)

  d_size, chans, samples = Features.shape
  
  model=ShallowConvNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = dropout_rate)

  # compile the model and set the optimizers
  model.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics = ['accuracy'])
  
  # count number of parameters in the model
  checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                                save_best_only=True)
  
  
  class_weights = {0:np.sum(y)/(len(y)-np.sum(y)), 1:1}
  
  fittedModel = model.fit(Features[train_index], Y[train_index], batch_size = 64, epochs =30 , 
                          verbose=2, validation_data=(Features[test_index], Y[test_index]),
                          callbacks=[checkpointer],class_weight=class_weights)
  probs       = model.predict(Features[test_index])
  preds       = probs.argmax(axis = -1)
  test = Y[test_index].argmax(axis=-1)
  acc = accuracy_score(test, preds)
  cm = confusion_matrix(test, preds)
  f1= f1_score(test, preds)
  return test, preds, acc,f1 , cm, model
  

In [ ]:
kf=KFold(n_splits=5,shuffle=True,random_state=0)

In [ ]:
train_index=list(kf.split(Features))[0][0]
test_index=list(kf.split(Features))[0][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 47s - loss: 0.7275 - accuracy: 0.6348 - val_loss: 0.5304 - val_accuracy: 0.7451

Epoch 00001: val_loss improved from inf to 0.53040, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5545 - accuracy: 0.7650 - val_loss: 0.4535 - val_accuracy: 0.7980

Epoch 00002: val_loss improved from 0.53040 to 0.45353, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4757 - accuracy: 0.8103 - val_loss: 0.3918 - val_accuracy: 0.8420

Epoch 00003: val_loss improved from 0.45353 to 0.39176, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4294 - accuracy: 0.8372 - val_loss: 0.3690 - val_accuracy: 0.8596

Epoch 00004: val_loss improved from 0.39176 to 0.36898, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 17s - loss: 0.4027 - accuracy: 0.8561 - val_loss: 0.3470 - val_accuracy: 0.8711

Epoch 00005: val_loss improved from 0.36898 to 0.34703, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 17s - loss: 0.3819

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 0, 1, ..., 0, 1, 0]),
 0.90625,
 0.9150743099787686,
 array([[2054,  162],
        [ 318, 2586]]),
 <keras.engine.functional.Functional at 0x7f53f07a4b10>)

In [ ]:
train_index=list(kf.split(Features))[1][0]
test_index=list(kf.split(Features))[1][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 18s - loss: 0.7327 - accuracy: 0.6321 - val_loss: 0.5393 - val_accuracy: 0.7301

Epoch 00001: val_loss improved from inf to 0.53927, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5537 - accuracy: 0.7634 - val_loss: 0.4467 - val_accuracy: 0.8105

Epoch 00002: val_loss improved from 0.53927 to 0.44670, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4702 - accuracy: 0.8123 - val_loss: 0.4302 - val_accuracy: 0.8076

Epoch 00003: val_loss improved from 0.44670 to 0.43016, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4292 - accuracy: 0.8410 - val_loss: 0.3822 - val_accuracy: 0.8459

Epoch 00004: val_loss improved from 0.43016 to 0.38218, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 17s - loss: 0.4085 - accuracy: 0.8517 - val_loss: 0.3581 - val_accuracy: 0.8705

Epoch 00005: val_loss improved from 0.38218 to 0.35814, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 17s - loss: 0.3844

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([0, 1, 1, ..., 1, 1, 1]),
 0.918359375,
 0.926096181046676,
 array([[2083,  224],
        [ 194, 2619]]),
 <keras.engine.functional.Functional at 0x7f53f078f210>)

In [ ]:
train_index=list(kf.split(Features))[2][0]
test_index=list(kf.split(Features))[2][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 19s - loss: 0.7191 - accuracy: 0.6427 - val_loss: 0.4949 - val_accuracy: 0.7668

Epoch 00001: val_loss improved from inf to 0.49487, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5371 - accuracy: 0.7745 - val_loss: 0.4382 - val_accuracy: 0.8020

Epoch 00002: val_loss improved from 0.49487 to 0.43821, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4620 - accuracy: 0.8169 - val_loss: 0.3785 - val_accuracy: 0.8477

Epoch 00003: val_loss improved from 0.43821 to 0.37855, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4254 - accuracy: 0.8399 - val_loss: 0.3702 - val_accuracy: 0.8525

Epoch 00004: val_loss improved from 0.37855 to 0.37021, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 17s - loss: 0.3943 - accuracy: 0.8565 - val_loss: 0.3402 - val_accuracy: 0.8768

Epoch 00005: val_loss improved from 0.37021 to 0.34024, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 17s - loss: 0.3819

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 1, 1]),
 array([1, 1, 1, ..., 0, 1, 1]),
 0.909375,
 0.9177013125221708,
 array([[2069,  206],
        [ 258, 2587]]),
 <keras.engine.functional.Functional at 0x7f53f07d9610>)

In [ ]:
train_index=list(kf.split(Features))[3][0]
test_index=list(kf.split(Features))[3][1]

In [ ]:
resuslt=run(train_index,test_index)


Epoch 00001: val_loss improved from inf to 0.54177, saving model to /tmp/checkpoint.h5

Epoch 00002: val_loss improved from 0.54177 to 0.46781, saving model to /tmp/checkpoint.h5

Epoch 00003: val_loss improved from 0.46781 to 0.39406, saving model to /tmp/checkpoint.h5

Epoch 00004: val_loss improved from 0.39406 to 0.36722, saving model to /tmp/checkpoint.h5

Epoch 00005: val_loss improved from 0.36722 to 0.35804, saving model to /tmp/checkpoint.h5

Epoch 00006: val_loss improved from 0.35804 to 0.33746, saving model to /tmp/checkpoint.h5

Epoch 00007: val_loss did not improve from 0.33746

Epoch 00008: val_loss improved from 0.33746 to 0.33484, saving model to /tmp/checkpoint.h5

Epoch 00009: val_loss did not improve from 0.33484

Epoch 00010: val_loss improved from 0.33484 to 0.31025, saving model to /tmp/checkpoint.h5

Epoch 00011: val_loss did not improve from 0.31025

Epoch 00012: val_loss did not improve from 0.31025

Epoch 00013: val_loss did not improve from 0.31025

Epoch 0

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 0.8908203125,
 0.9026302037972479,
 array([[1970,  328],
        [ 231, 2591]]),
 <keras.engine.functional.Functional at 0x7fd52015fa50>)

In [ ]:
train_index=list(kf.split(Features))[4][0]
test_index=list(kf.split(Features))[4][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 18s - loss: 0.7255 - accuracy: 0.6368 - val_loss: 0.5378 - val_accuracy: 0.7281

Epoch 00001: val_loss improved from inf to 0.53782, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5624 - accuracy: 0.7561 - val_loss: 0.4483 - val_accuracy: 0.8002

Epoch 00002: val_loss improved from 0.53782 to 0.44832, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4745 - accuracy: 0.8153 - val_loss: 0.3856 - val_accuracy: 0.8504

Epoch 00003: val_loss improved from 0.44832 to 0.38563, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4293 - accuracy: 0.8396 - val_loss: 0.3809 - val_accuracy: 0.8596

Epoch 00004: val_loss improved from 0.38563 to 0.38087, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 17s - loss: 0.4106 - accuracy: 0.8501 - val_loss: 0.3573 - val_accuracy: 0.8689

Epoch 00005: val_loss improved from 0.38087 to 0.35728, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 17s - loss: 0.3905

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 0.899609375,
 0.9076204169662114,
 array([[2081,  263],
        [ 251, 2525]]),
 <keras.engine.functional.Functional at 0x7fd48a5f6e90>)

In [ ]:
results=[]
for train_index, test_index in kf.split(Features):
  res=run(train_index,test_index)
  results.append(res)
  print(res)

Epoch 1/30
320/320 - 18s - loss: 0.7240 - accuracy: 0.6426 - val_loss: 0.5208 - val_accuracy: 0.7592

Epoch 00001: val_loss improved from inf to 0.52084, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5542 - accuracy: 0.7589 - val_loss: 0.4520 - val_accuracy: 0.8010

Epoch 00002: val_loss improved from 0.52084 to 0.45199, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4749 - accuracy: 0.8104 - val_loss: 0.4008 - val_accuracy: 0.8449

Epoch 00003: val_loss improved from 0.45199 to 0.40076, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4250 - accuracy: 0.8417 - val_loss: 0.3654 - val_accuracy: 0.8625

Epoch 00004: val_loss improved from 0.40076 to 0.36542, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 16s - loss: 0.4006 - accuracy: 0.8557 - val_loss: 0.3770 - val_accuracy: 0.8490

Epoch 00005: val_loss did not improve from 0.36542
Epoch 6/30
320/320 - 17s - loss: 0.3849 - accuracy: 0.8673 - val_loss: 0.3460 -